In [1]:
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
'''Set the constant paths here'''
root = "/home/akmyths/Project/Data"
path = "/home/akmyths/Project/Data/Resized_final_data"
csv_path = "/home/akmyths/Project/Data/train_info.csv"

In [3]:
pickle_files_names = ("Data_final.pickle",)
Formatted_Data = []
for Dfile in pickle_files_names:
    with open(os.path.join(root, Dfile), "rb") as f:
        Formatted_Data.append(pickle.load(f))

In [4]:
len(Formatted_Data)

1

In [5]:
# recollect the fragmented data

labels_mappings = Formatted_Data[0]['label_mapping']
print(labels_mappings)

train_dataSet = Formatted_Data[0]['train_data']
train_labels  = Formatted_Data[0]['train_labels']

cv_dataSet = Formatted_Data[0]['cv_data']
cv_labels = Formatted_Data[0]['cv_labels']

test_dataSet = Formatted_Data[0]['test_data']
test_labels  = Formatted_Data[0]['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'abstract': 0, 'history painting': 13, 'marina': 18, 'design': 9, 'portrait': 22, 'landscape': 16, 'symbolic painting': 29, 'allegorical painting': 2, 'miniature': 19, 'sketch and study': 27, 'animal painting': 3, 'nude painting (nu)': 21, 'flower painting': 11, 'battle painting': 4, 'poster': 23, 'interior': 15, 'cloudscape': 8, 'mythological painting': 20, 'genre painting': 12, 'shan shui': 26, 'yakusha-e': 32, 'literary painting': 17, 'illustration': 14, 'caricature': 6, 'advertisement': 1, 'cityscape': 7, 'religious painting': 24, 'bird-and-flower painting': 5, 'still life': 28, 'figurative': 10, 'self-portrait': 25, 'veduta': 30, 'wildlife painting': 31}
(516, 100, 100, 3) (74, 100, 100, 3) (148, 100, 100, 3)
(516, 33) (74, 33) (148, 33)


In [6]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [13]:
batch_size = 16
patch_size = 5
depth = 16 # number of filters in a layer
num_hidden = 256

image_size = 100
num_labels = 33
num_channels = 3 # rgb images

In [14]:
graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
    tf_valid_dataset = tf.constant(cv_dataSet)
    tf_test_dataset = tf.constant(test_dataSet)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [15]:
num_steps = 1501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
    
    print('Cross_Validation accuracy: %.1f%%' % accuracy(validation_prediction.eval(), cv_labels))

Initialized
Minibatch loss at step 0: 8.764420
Minibatch accuracy: 6.2%
Minibatch loss at step 50: 3.087005
Minibatch accuracy: 37.5%
Minibatch loss at step 100: 3.655488
Minibatch accuracy: 31.2%
Minibatch loss at step 150: 3.925866
Minibatch accuracy: 6.2%
Minibatch loss at step 200: 3.291370
Minibatch accuracy: 18.8%
Minibatch loss at step 250: 5.081890
Minibatch accuracy: 6.2%
Minibatch loss at step 300: 3.613737
Minibatch accuracy: 0.0%
Minibatch loss at step 350: 2.863400
Minibatch accuracy: 31.2%
Minibatch loss at step 400: 4.101360
Minibatch accuracy: 6.2%
Minibatch loss at step 450: 5.040809
Minibatch accuracy: 6.2%
Minibatch loss at step 500: 4.183412
Minibatch accuracy: 6.2%
Minibatch loss at step 550: 4.522226
Minibatch accuracy: 0.0%
Minibatch loss at step 600: 2.635675
Minibatch accuracy: 31.2%
Minibatch loss at step 650: 3.791253
Minibatch accuracy: 6.2%
Minibatch loss at step 700: 4.532957
Minibatch accuracy: 6.2%
Minibatch loss at step 750: 3.524948
Minibatch accuracy: